# Exploring Wikipedia clickstream data: English Wiki in December 2018    

## Graph modeling

### 1. Introduction  

This notebook contains graph modeling and data import steps for the English Wikipedia clickstream dataset for December 2018.  
This is the 3rd part of a project about the usage patterns of Wikipedia.  
The preceding parts are:  
1. [Data quality analysis of available datasets](data_quality_analysis.ipynb)
2. [Exploratory data analysis of the English Wikipedia clickstream dataset for December 2018](English_Wikipedia_EDA.ipynb)

In this notebook, we'll use the insights from the exploratory data analysis of our clickstream dataset to model that data as a graph, and then we'll import the data into a neo4j graph database according to our graph model.

#### Notebook contents:  

1. [Introduction](#1.-Introduction)  
2. [Notebook setup](#2.-Notebook-setup)  
3. [Graph modeling](#3.-Graph-modeling)
4. [Neo4j setup on AWS EC2 Ubuntu](#4.-Neo4j-setup-on-AWS-EC2-Ubuntu)   
5. [Import data into neo4j](#5.-Import-data-into-neo4j)  
   5.1 [Get the clickstream data onto AWS](#5.1-Get-the-clickstream-data-onto-AWS)  
   5.2 [Connect this notebook to neo4j](#5.2-Connect-this-notebook-to-neo4j)  
   5.3 [Set indices](#5.3-Set-indices)  
   5.4 [Load in the data](#5.4-Load-in-the-data)  
   5.5 [Check the loaded database](#5.5-Check-the-loaded-database)  
   5.6 [Adress outliers](#5.6-Adress-outliers)  
6. [Summary](#6.-Summary)
7. [Next steps](#7.-Next-steps)

### 2. Notebook setup  

#### Imports

In [1]:
import pandas as pd
import numpy as np

import os

import  csv

from time import sleep
from timeit import default_timer as timer

# custom general helper functions for this project
import custom_utils as cu
import importlib

In [49]:
# reload imports as needed
importlib.reload(cu);

#### Settings

In [2]:
# set up Pandas options
pd.set_option('display.max_columns', 25)
pd.set_option('display.max_rows', 100)
pd.set_option('display.precision', 3)
pd.options.display.float_format = '{:.2f}'.format

### 3. Graph modeling  

In our dataset, we have Wikipedia articles and external referers with directed clickstream events connecting them, and we also have the aggregated traffic volumes for those events. In a graph model, we can represent the Wikipedia articles as nodes, and the clickstream events as directed edges. The incoming traffic volume from external referers can be captured as node properties, and the clickstream traffic volumes can be captured as edge properties.  

We'll have two types of relationships/edges in the model: LINK_TO and SEARCH_FOR, representing link-based clickstream events and internal search clickstream events. Since internal search clickstream events are just clickstream events from one article to the other without the corresponding link between them, it is impossible to have a LINK_TO and a SEARCH_FOR relationships going in the same direction between two nodes (though they can go in the opposite directions).

We can make a quick sketch of our graph model using this neat [arrows tool](http://www.apcjones.com/arrows/) for sketching graphs:  

<img src="img/arrows_articles_graph_sketch.png" alt="Graph model sketch" width="65%">


So, our Wikipedia clickstream graph model consists of:  
- **node**: Article  
  - node properties: article title, incoming external traffic volumes by external referer type
- **relationships/edges**: LINK_TO and SEARCH_FOR
  - both relationships are directed  
  - relationship property: traffic volume  
- **special cases**: we'll need to adress some caveats for special Wikipedia articles like "Main Page", "Hyphen-minus", etc.

### 4. Neo4j setup on AWS EC2 Ubuntu

#### Install Neo4j community edition server  

- For installing neo4j on an Ubuntu EC2 instance on AWS, follow the Debian installation instructions [here](https://neo4j.com/docs/operations-manual/current/installation/linux/debian/). 
- [Run neo4j as a service using `systemctl`](https://neo4j.com/docs/operations-manual/current/installation/linux/systemd/#linux-service-control):  
  - To start neo4j: `systemctl start neo4j`  
  - To stop neo4j: `systemctl stop neo4j`

- run neo4j and tunnel into the neo4j browser:
  - run neo4j: `systemctl start neo4j`
  - tunnel into the browser, e.g.: `ssh -NfL localhost:7474:localhost:7474 -L localhost:7687:localhost:7687 arinai@<AWS instance public IP address>`
  - in a browser, open `http://localhost:7474/browser/`
  - On first login, both the username and password are `neo4j`. After the initial login there will be a prompt to set a new password. 

### 5. Import data into neo4j

#### 5.1 Get the clickstream data onto AWS

Since we already have cleaned the clickstream tsv datafile in the exploratory data analysis notebook, we can upload it from local machine onto AWS with `scp` in terminal, e.g.:  
```
scp ~/projects/wiki_graph/data/clickstream-enwiki-2018-12_clean.tsv arinai@<AWS instance public IP address>:/var/lib/neo4j/import/  
```
However, this took about 15 min to run, and it would probably be much faster to `wget` the datafile from the [Wikimedia datadump](https://dumps.wikimedia.org/other/clickstream/) and run [the cleaning `sed` command](data_quality_analysis.ipynb#Escape-backslashes-first,-and-then-double-quotes) on it on AWS.

**Note:** By default, neo4j only imports data from its own `import` subfolder (on Ubuntu, it is `/var/lib/neo4j/import`) or from an url, so make sure to copy/move the data into the neo4j import folder (alternatively, this requirement can be commented out in the `conf` file).

In [10]:
# Check the top rows of the tsv file
!head /var/lib/neo4j/import/clickstream-enwiki-2018-12_clean.tsv

other-empty	2019_Horizon_League_Baseball_Tournament	external	16
other-search	ForeverAtLast	external	40
other-empty	ForeverAtLast	external	85
First_Families_of_Pakistan	Jehangir_Wadia	link	19
The_Lawrence_School,_Sanawar	Jehangir_Wadia	link	36
Wadia_family	Jehangir_Wadia	link	715
other-search	Jehangir_Wadia	external	967
Ness_Wadia	Jehangir_Wadia	link	494
other-empty	Jehangir_Wadia	external	638
GoAir	Jehangir_Wadia	link	1191


#### 5.2 Connect this notebook to neo4j  

We can use the `py2neo` library to connect from python to a neo4j instance.

In [3]:
from py2neo import authenticate, Graph, Node, Relationship

To avoid typing the neo4j password into the notebook each time, we can save it in a separate file and read it in with the helper function. There are many ways to do this.  
For this project, we'll create a file called `.n4jpass` in the current folder, with the following contents:  
```
# comments here
user=neo4j
password=secretStuff123
``` 
(Just replace the password in the above with your actual neo4j password.)  

The helper function to read in the neo4j username and password is called `read_n4jpass`, and it's in the `custom_utils.py` module in the current folder. 

In [5]:
# read in the neo4j credentials from .n4jpass
n4j_cred = read_n4jpass()

In [6]:
# set up authentication parameters
authenticate("localhost:7474", n4j_cred["user"], n4j_cred["password"])

In [7]:
# connect to authenticated graph database
graph = Graph("http://localhost:7474/db/data/")

#### 5.3 Set indices  

**Note:** do this step BEFORE importing the data!  

Unlike relational databases, there are no built-in constraints or automatic indexing in neo4j. Since we expect Wikipedia article names to be unique, let's add a uniqueness constraint. This will make the import queries run faster.

In [20]:
graph.run("CREATE CONSTRAINT ON (a:Article) ASSERT a.title IS UNIQUE;");

In [68]:
# list all indices in the db
r = graph.data('CALL db.indexes;')
pd.DataFrame(r)

,description,failureMessage,id,indexName,progress,properties,provider,state,tokenNames,type
0,INDEX ON :Article(title),,1,index_1,100.0,[title],"{'version': '1.0', 'key': 'native-btree'}",ONLINE,[Article],node_unique_property


#### 5.4 Load in the data  

There are a few ways to load data into neo4j. Here, we'll use `LOAD CSV` with `PERIODIC COMMIT` to load the data.  

##### Check data read-in  
The `LOAD CSV` statement allows us to perform some data parsing at load time. We can check what the incoming data rows look like to the `LOAD CSV` statement and whether the data is being parsed correctly by using the `LOAD CSV` with `RETURN` and `LIMIT` statements, without writing anything to the database yet.

In [73]:
filename = "clickstream-enwiki-2018-12_clean.tsv"

In [74]:
fp = 'file:///' + filename
    
query_test = """
    LOAD CSV FROM {myfilepath} AS row
    FIELDTERMINATOR '\t'
    RETURN row
    LIMIT 10
    ;
    """

test = graph.data(query_test, myfilepath=str(fp))
pd.DataFrame(test)

,row
0,"[other-empty, 2019_Horizon_League_Baseball_Tou..."
1,"[other-search, ForeverAtLast, external, 40]"
2,"[other-empty, ForeverAtLast, external, 85]"
3,"[First_Families_of_Pakistan, Jehangir_Wadia, l..."
4,"[The_Lawrence_School,_Sanawar, Jehangir_Wadia,..."
5,"[Wadia_family, Jehangir_Wadia, link, 715]"
6,"[other-search, Jehangir_Wadia, external, 967]"
7,"[Ness_Wadia, Jehangir_Wadia, link, 494]"
8,"[other-empty, Jehangir_Wadia, external, 638]"
9,"[GoAir, Jehangir_Wadia, link, 1191]"


In [75]:
query_test = """
    LOAD CSV FROM {myfilepath} AS row
    FIELDTERMINATOR '\t'
    RETURN row[0], row[1], split(row[0], '-')[1], toInteger(row[3])
    LIMIT 10
    ;
    """

test = graph.data(query_test, myfilepath=str(fp))
pd.DataFrame(test)

,row[0],row[1],"split(row[0], '-')[1]",toInteger(row[3])
0,other-empty,2019_Horizon_League_Baseball_Tournament,empty,16
1,other-search,ForeverAtLast,search,40
2,other-empty,ForeverAtLast,empty,85
3,First_Families_of_Pakistan,Jehangir_Wadia,None,19
4,"The_Lawrence_School,_Sanawar",Jehangir_Wadia,None,36
5,Wadia_family,Jehangir_Wadia,None,715
6,other-search,Jehangir_Wadia,search,967
7,Ness_Wadia,Jehangir_Wadia,None,494
8,other-empty,Jehangir_Wadia,empty,638
9,GoAir,Jehangir_Wadia,None,1191


##### Load nodes first  

When edges are being created in neo4j, they either connect existing nodes or create any non-existing nodes that are part of the edges, which can be very slow. Loading just the nodes first can significantly speed up the overall data import.  

The English Wikipedia dataset is very large, and loading it into neo4j from this notebook using `py2neo` turned out to be too slow. Instead, we'll load the data using `cypher-shell` in terminal.

###### cypher-shell queries  

These queries were run by running `cypher-shell` in terminal on AWS, logging in with neo4j credentials, and copying+pasting the scripts below, one at a time. A less manual way to run these queries is to save them to a text file and pipe them to `cypher-shell`.  

Because the raw data file is very large, the nodes/edges read-ins are split up by reference types to create smaller file chunks, to make it easier to test the results.  

**External reference type subsets, by referer category** 
```
USING PERIODIC COMMIT
    LOAD CSV FROM "file:///clickstream-enwiki-2018-12_clean.tsv" AS row
    FIELDTERMINATOR '\t'
    WITH row
    WHERE row[0] = 'other-empty' 
    CREATE (n:Article { title: row[1], empty_referer_traffic:  toInteger(row[3]) })
    ;
```  
_Added 5093433 nodes, Set 10186866 properties, Added 5093433 labels_  
The above ran for about 5 min in terminal cypher-shell on AWS.  

```
USING PERIODIC COMMIT
    LOAD CSV FROM "file:///clickstream-enwiki-2018-12_clean.tsv" AS row
    FIELDTERMINATOR '\t'
    WITH row
    WHERE row[0] = 'other-external' 
    MERGE (n:Article { title: row[1] })
    ON CREATE SET n.external_website_traffic = toInteger(row[3])
    ON MATCH SET n.external_website_traffic = toInteger(row[3])
    ;
```  
_Added 726 nodes, Set 788037 properties, Added 726 labels_  
The above ran for about 2 min in terminal cypher-shell on AWS.  

```
USING PERIODIC COMMIT
    LOAD CSV FROM "file:///clickstream-enwiki-2018-12_clean.tsv" AS row
    FIELDTERMINATOR '\t'
    WITH row
    WHERE row[0] = 'other-other' 
    MERGE (n:Article { title: row[1] })
    ON CREATE SET n.unknown_external_traffic = toInteger(row[3])
    ON MATCH SET n.unknown_external_traffic = toInteger(row[3])
    ;
```  
_Added 31 nodes, Set 374952 properties, Added 31 labels_  
The above ran for about 2 min in terminal cypher-shell on AWS.  

```
USING PERIODIC COMMIT
    LOAD CSV FROM "file:///clickstream-enwiki-2018-12_clean.tsv" AS row
    FIELDTERMINATOR '\t'
    WITH row
    WHERE row[0] = 'other-internal' 
    MERGE (n:Article { title: row[1] })
    ON CREATE SET n.other_wikimedia_traffic = toInteger(row[3])
    ON MATCH SET n.other_wikimedia_traffic = toInteger(row[3])
    ;
```  
_Added 3454 nodes, Set 1352346 properties, Added 3454 labels_  
The above ran for about 2 min in terminal cypher-shell on AWS.  

```
USING PERIODIC COMMIT
    LOAD CSV FROM "file:///clickstream-enwiki-2018-12_clean.tsv" AS row
    FIELDTERMINATOR '\t'
    WITH row
    WHERE row[0] = 'other-search' 
    MERGE (n:Article { title: row[1] })
    ON CREATE SET n.external_search_traffic = toInteger(row[3])
    ON MATCH SET n.external_search_traffic = toInteger(row[3])
    ;
```  
_Added 65465 nodes, Set 3451890 properties, Added 65465 labels_  
The above ran for about 3.5 min in terminal cypher-shell on AWS.



**Non-external reference type subset**
```
USING PERIODIC COMMIT
    LOAD CSV FROM "file:///clickstream-enwiki-2018-12_clean.tsv" AS row
    FIELDTERMINATOR '\t'
    WITH row
    WHERE row[2] <> 'external' 
    MERGE (n1:Article { title: row[0] })
    MERGE (n2:Article { title: row[1] })
    ;
```  
_Added 22590 nodes, Set 22590 properties, Added 22590 labels_  
The above ran for about 13 min in terminal cypher-shell on AWS.

##### Load edges/relationships  
In graphs, edges=relationships.

**LINK_TO relationships**  
```
USING PERIODIC COMMIT
    LOAD CSV FROM "file:///clickstream-enwiki-2018-12_clean.tsv" AS row
    FIELDTERMINATOR '\t'
    WITH row
    WHERE row[2] = 'link' 
    MATCH (n1:Article { title: row[0] })
    MATCH (n2:Article { title: row[1] })
    CREATE (n1)-[r:LINK_TO { traffic: toInteger(row[3]) }]->(n2)
    ;
```  
_Created 17851574 relationships, Set 17851574 properties_  
The above ran for about 26 min in terminal cypher-shell on AWS.


**SEARCH_FOR relationships**  
```
USING PERIODIC COMMIT
    LOAD CSV FROM "file:///clickstream-enwiki-2018-12_clean.tsv" AS row
    FIELDTERMINATOR '\t'
    WITH row
    WHERE row[2] = 'other' 
    MATCH (n1:Article { title: row[0] })
    MATCH (n2:Article { title: row[1] })
    CREATE (n1)-[r:SEARCH_FOR { traffic: toInteger(row[3]) }]->(n2)
    ;
```  
_Created 1005180 relationships, Set 1005180 properties_  
The above ran for about 14 min in terminal cypher-shell on AWS.  


Total runtime was about 70 min on AWS cypher-shell. Every other method of import was much much slower.

#### 5.5 Check the loaded database  

Cypher queries for inspecting the loaded data in the neo4j browser. The visualizations are exported from neo4j browser.

Visualize the database schema  
```
call db.schema.visualization()  
```  
Output:

<div>
<img src="img/EN_wiki_1218_schema.png" alt="neo4j db schema" width="20%" style="margin-left: 0;">
</div>

#### 5.6 Adress outliers  

Fix any article linkings that likely do not represent the user behavior we're trying to model.  
The queries below were run and visualized in neo4j browser.

##### "Main Page" article out-links  
```
match p=(a:Article)-[r:LINK_TO]->()
where a.title="Main_Page"
return p
limit 200  
```  
Output:

<div>
<img src="img/en_1218_Main_Page_outgoing_links_graph.png" alt="Main Page out-links graph" width="60%" style="margin-left: 0;">
</div>

##### "Main Page" in-links
```
match p=(a:Article)<-[r:LINK_TO]-()
where a.title="Main_Page"
return p
limit 200  
```  
Output:

<div>
<img src="img/en_1218_Main_Page_incoming_links_graph.png" alt="Main Page in-links graph" width="60%" style="margin-left: 0;">
</div>

##### "Main Page" out-searches
```
match p=(a:Article)-[r:SEARCH_FOR]->()
where a.title="Main_Page"
return p
limit 200  
```
Output:

<div>
<img src="img/en_1218_Main_Page_outgoing_search_graph_lim200.png" alt="Main Page out-searches graph" width="60%" style="margin-left: 0;">
</div>

##### "Main Page" in-searches  
```
match p=(a:Article)<-[r:SEARCH_FOR]-()
where a.title="Main_Page"
return p
limit 200  
```
Output:

<div>
<img src="img/en_1218_Main_Page_incoming_search_graph_lim200.png" alt="Main Page in-searches graph" width="60%" style="margin-left: 0;">
</div>

**Conclusion:** Keep the link traffic to the "Main Page" article, because it seems to be focused on a specific topic and does not overwhelm the graph, but drop the link traffic from "Main Page" and the to/from search traffic, because those relationships are likely to overlink any topic-specific browsing behaviors. We'll save the dropped "Main Page" article traffic values on nodes.  


cypher-shell query to remove Main_Page out-link-traffic:  
```
MATCH (mp:Article {title: "Main_Page"})-[r:LINK_TO]->(a:Article)
SET a.link_traffic_from_main_page = r.traffic
DELETE r;  
```  
_0 rows available after 141 ms, consumed after another 0 ms  
Deleted 73 relationships, Set 73 properties_  


cypher-shell query to remove Main_Page out-search-traffic:  
```
MATCH (mp:Article {title: "Main_Page"})-[r:SEARCH_FOR]->(a:Article)
SET a.search_traffic_from_main_page = r.traffic
DELETE r;  
```   
_0 rows available after 30816 ms, consumed after another 0 ms  
Deleted 257794 relationships, Set 257794 properties_  


cypher-shell query to remove Main_Page in-search-traffic:  
```
MATCH (mp:Article {title: "Main_Page"})<-[r:SEARCH_FOR]-(a:Article)
SET a.search_traffic_to_main_page = r.traffic
DELETE r;  
```  
_0 rows available after 1427 ms, consumed after another 0 ms  
Deleted 110009 relationships, Set 110009 properties_

##### "Hyphen-minus" article out-searches  
```
match p=(a:Article)-[r:SEARCH_FOR]->()
where a.title="Hyphen-minus"
return p
limit 200
```
The only out-search from "Hyphen-minus" was to the "Main Page" article, which was deleted in the queries above.

##### "Hyphen-minus" in-searches  
```
match p=(a:Article)<-[r:SEARCH_FOR]-()
where a.title="Hyphen-minus"
return p
limit 200
```  

Output:  

<div>
<img src="img/en_1218_Hyphen_minus_incoming_search_graph_lim200.png" alt="Hyphen-minus in-searches graph" width="60%" style="margin-left: 0;">
</div>

**Conclusion:** Drop the incoming search traffic to the "Hyphen-minus" article. We probably don't need to save the dropped "Hyphen-minus" traffic values on nodes, but doing it here just in case it looks interesting later.  

cypher-shell query to remove "Hyphen-minus" in-search-traffic:  
```
MATCH (hm:Article {title: "Hyphen-minus"})<-[r:SEARCH_FOR]-(a:Article)
SET a.search_traffic_to_hyphen_minus = r.traffic
DELETE r;  
```  
_0 rows available after 1569 ms, consumed after another 0 ms  
Deleted 127457 relationships, Set 127457 properties_

##### "Undefined" article out-searches  
```
match p=(a:Article)-[r:SEARCH_FOR]->()
where a.title="Undefined"
return p
limit 200
```  
The only out-search from the "Undefined" article was to the "Main Page" article, which was deleted in the queries above.  

##### "Undefined" article in-searches  
```
match p=(a:Article)<-[r:SEARCH_FOR]-()
where a.title="Undefined"
return p
limit 200
```  
Output:  

<div>
<img src="img/en_1218_Undefined_incoming_search_graph_lim200.png" alt="Undefined in-searches graph" width="60%" style="margin-left: 0;">
</div>

**Conclusion:** Drop the incoming search traffic to the "Undefined" article. We probably don't need to save the dropped "Undefined" traffic values on nodes, but doing it here just in case it looks interesting later.  

cypher-shell query to remove "Undefined" in-search-traffic:  
```
MATCH (u:Article {title: "Undefined"})<-[r:SEARCH_FOR]-(a:Article)
SET a.search_traffic_to_undefined = r.traffic
DELETE r;  
```  
_0 rows available after 37 ms, consumed after another 0 ms
Deleted 241 relationships, Set 241 properties_

Let's get a sample of the resulting graph paths in neo4j browser:  
```
MATCH p=()-[]->() RETURN p LIMIT 300
```  
Output:  

<div>
<img src="img/en_1218_final_graph_sample300.png" alt="Final graph sample" width="90%" style="margin-left: 0;">
</div>

The graph sample above looks good, with nodes forming clusters, and no overlinking overwhelming the graph.  
This Wikipedia clickstream graph can now be used to find influential articles and detect article communities.

### 6. Summary  

In this notebook, we've defined the graph model for the English Wikipedia clickstream data, set up a neo4j graph database on AWS EC2, and imported the data according to the graph model.  

### 7. Next steps   

Now that we have the clickstream data stored as a graph, in the [next notebook](English_Wikipedia_network_analysis_AWS.ipynb) we'll use network analysis tenchniques to describe the relationship structures within the data, identify infuential nodes and detect article communities via user browsing behavior. 